<a href="https://colab.research.google.com/github/MayerT1/ForestStandHeight/blob/master/Exercise2_Install_ISCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install ISCE using Colaboratory


#What is ISCE?

ISCE stands for “InSAR Scientific Computing Environment software.” ISCE supports processing of Interferometric Synthetic Aperture Radar from a several space-borne satellites and one airborne sensor. 

#Why use ISCE?

We will use ISCE to preprocess create the interferograms we need for the FSH algorithm from ALOS PALSAR Single Look Complex (SLC) imagery. Originally funded by NASA's Earth Science Technology Office (ESTO) under the Advanced Information Systems Technology (AIST) 2008, ISCE is currently funded under the NASA-ISRO SAR (NISAR) project. Thus, this is a great tool to start using for SAR preprocessing.

#Useful Links

* ISCE-2 [GitHub](https://github.com/isce-framework/isce2)
* ISCE-2 [Conda-Forge](https://anaconda.org/conda-forge/isce2)
* ISCE Users [Forum](http://earthdef.caltech.edu/projects/isce_forum/boards/4)
* [Official Website](https://winsar.unavco.org/software/isce)
* UNAVCO [Video](https://zoom.us/recording/play/lIm3JJqpElPuuvrXvFwy3PUnkc5rWnK0lyagG6DeAgbu_ESSZT9_kUgh1T39ssPm?startTime=1564416529000) on Installation Using Homebrew or Conda-forge

#Installation Flow

![alt text](https://drive.google.com/uc?id=1amhhLiGKaIx4md-9gxvJNWVPaunwm4d-)

In order to install ISCE, we will set up the Colaboratory virtual machine (VM). We will clone the required files from the ISCE-2 GitHub page into our colaboratory. Then we will install ISCE using SCons. SCons will run for about an hour before the Runtime Disconnected error appears. After closing and reopening the GitHub, we will move the `Helper` folder created during the SCons install. Then we will set required environmental varaibles and finally test the ISCE installation by calling `stripmapApp.py`, the function we will be using to preprocess our ALOS PALSAR data.
 

# 1. Set Up Colaboratory Virtual Machine

Before installing ISCE, we will set up the  Virtual Machine where the code in this Colaboratory will be executed. Set up includes mounting your Google Drive and installing dependencies required by the ISCE software. 

##1.1 Mount Google Drive

In order to access your Google Drive and all of the files stored within from the colab, you need to mount your drive. When you run the following code, you will be prompted to authorize your Google account. In this section, you are also creating a symbolic pathname `content/notebooks` that represents `content/mnt/My Drive/Colab Notebooks` to circumvent issues arising from the space in `My Drive`.

In [0]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

##1.2 Install Dependencies


Install all the relevant packages for running ISCE here. 

In [0]:
!apt update
!apt install -y gfortran libmotif-dev libhdf5-dev libfftw3-dev libgdal-dev scons python3 cython3 python3-scipy python3-matplotlib python3-h5py python3-gdal python3-pip

#2. Install ISCE

Now that our VM is set up, we will install ISCE. This includes cloning the current version of ISCE from the official ISCE GitHub page and installing using [SCons](https://scons.org/), an open source software construction tool. Since the SCons tool calls itself recursively, **the `helper` folder created during the SCons install process must be moved after running SCons**. 

##2.1 Clone ISCE Files

Run the lines below to create a folder named `isce-notebook` within your colaboratory notebook folder on Google Drive. The final file structure will be `/content/notebooks/isce-notebook/`. This folder is where we will clone the files needed to install ISCE from GitHub. 

Use `!ls` to view the files in your current directory. `notebooks` is your Colab notebooks folder. 

In [0]:
!ls

Navigate into your notebooks folder.

In [0]:
%cd /content/notebooks/

Make a folder named `isce-notebook` to store the ISCE installation files.

In [0]:
!mkdir isce-notebook

Navigate into the `isce-notebook` folder. 

In [0]:
%cd /content/notebooks/isce-notebook/

Clone the ISCE files from GitHub to the `isce-notebook` folder. This should only take a few moments.

In [0]:
!git clone https://github.com/isce-framework/isce2.git

You should now have a file structure as shown below. Notice that the `isce2` folder and all of its contents are now within your `isce-notebook` folder.

![alt text](https://drive.google.com/uc?id=16-SNY-xYQ7p7U_q3ng3FMYlwFJjiGtPn)

##2.2 Create SConfigISCE File

Then, run the following code block to create the `SConfigISCE` file within the isce-notebook folder. This file builds paths necessary for building ISCE. 

In [0]:
from google.colab import files

with open('isce2/SConfigISCE', 'w') as f:
  f.write("""
  PRJ_SCONS_BUILD=/content/notebooks/isce-notebook/isce-2.2.0/build
  PRJ_SCONS_INSTALL=/content/notebooks/isce-notebook/isce-2.2.0/install/isce

  LIBPATH=/usr/lib/x86_64-linux-gnu /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial
  CPPPATH=/usr/include/x86_64-linux-gnu /usr/include /usr/include/python3.6m /usr/include/hdf5/serial /usr/include/gdal
  FORTRANPATH=/usr/include /usr/lib/gcc/x86_64-linux-gnu/7/finclude

  FORTRAN=/usr/bin/gfortran
  CC=/usr/bin/gcc
  CXX=/usr/bin/g++

  MOTIFLIBPATH = /usr/lib/x86_64-linux-gnu
  X11LIBPATH = /usr/lib/x86_64-linux-gnu
  MOTIFINCPATH = /usr/include/Xm
  X11INCPATH = /usr/include/X11

  ENABLE_CUDA=False
  """)

Then, navigate into the `isce2` folder cloned from GitHub. This is where the `SConfigISCE` file was created and where ISCE will be installed.

In [0]:
%cd isce2/
!ls

Install ISCE using the SCons command. This will use the information in the `SConfig` set up file to install ISCE. This will run for about an hour before throwing the **Runtime Disconnect** error shown below. Once the Colaboratory times out, **close the colab completely**, then reopen. Once you've reopened the colab, check to make sure your Google Drive is still connected. If it is not, rerun the step **1.1 Mount Google Drive**. 

![alt text](https://drive.google.com/uc?id=1prQ53gV1MHKfaoagSoZR2zLL7f0qdn9N)

In [0]:
import os, sys
os.environ['SCONS_CONFIG_DIR'] = "/content/notebooks/isce-notebook/isce2"
os.environ['PYTHONPATH'] = "/content/notebooks/isce-notebook/isce2/configuration"
!scons install --warn=no-all --setupfile=SConfigISCE

You should now have the file structure as shown below. Notice that there is now an `isce-2.2.0 `folder with a `helper` folder within `isce-2.2.0/build`. 

![alt text](https://drive.google.com/uc?id=16hi3WmBcLqvtGe4gcX_uGJjJzroZWmpl)

##2.3 Move `helper` Folder

Since the SCons tool calls itself recursively, we must move `helper` manually. Within your table of contents, navigate to `notebooks/isce-notebook/isce-2.2.0/build/`. Select the `helper` folder and move it to the `notebooks/isce-notebook/isce-2.2.0/install/isce` folder. You should now have the file structure as shown below.  

![alt text](https://drive.google.com/uc?id=1rKDH89ZHxfeIK3YxIEwYpNtnlKjQdrzU)

#3. Run ISCE

Now let's run ISCE! We will set the required environment variables and then test the ISCE installation.

##3.1 Set Environment Variables

ISCE needs four environment variables set in order to run. 

In [0]:
import os
os.environ['ISCE_ROOT'] = "/content/notebooks/isce-notebook/isce-2.2.0/install"
os.environ['ISCE_HOME'] = "/content/notebooks/isce-notebook/isce-2.2.0/install/isce"
os.environ['PATH'] = "/content/notebooks/isce-notebook/isce-2.2.0/install/isce/bin:/content/notebooks/isce-notebook/isce-2.2.0/install/isce/applications:/usr/bin/env:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin"
os.environ['PYTHONPATH']= "/content/notebooks/isce-notebook/isce-2.2.0/install:/content/notebooks/isce-notebook/isce-2.2.0/install/isce/applications:/content/notebooks/isce-notebook/isce-2.2.0/install/isce/components"

Use the personal working directory to confirm that you are within the `isce-2 `folder.

In [0]:
!pwd

If you aren't within the isce2 folder,  please use the change directory command below to navigate there. 

In [0]:
%cd notebooks/isce-notebook/isce2/

Check your environment `$PATH` variables.

In [0]:
!echo $PATH

##3.2 Test the ISCE installation

For this exercise, we will use `stripmapApp.py` to create interferograms from Single Look Complex (SLC) scenes from ALOS PALSAR. You can review the data and objectives of Exercise 2 [here](https://docs.google.com/presentation/d/1BP_EC6AZ4wonHluoIeoP_R-ChchMf970Il1Ie70gZJg/edit?usp=sharing). Let's check our installation of ISCE and learn more by calling `help` for `stripmapApp.py`.

In [0]:
!python3 $ISCE_HOME/applications/stripmapApp.py --help

After running the code block above, you should see a brief description of the ISCE software and InSAR application, in addition to a table of configurable parameters as show below.

![alt text](https://drive.google.com/uc?id=1YUmSHl2XL3n2cDb7PlPVS5pmSbuI9IjC)

#4. Next Steps

Next we will preprocess Single Look Complex (SLC) data from ALOS PALSAR using ISCE. Our output will be a geocoded interferogram. For this step, please continue to the Exercise2_Preprocess_ISCE Colab.